In [2]:
from libraries.db import DatabaseProxy
from bertopic import BERTopic
import dotenv
from bertopic.representation import OpenAI, KeyBERTInspired
import openai
import os
import pd

In [3]:
db = DatabaseProxy()
dotenv.load_dotenv(".envrc")

True

# Basic Statistics

Number of questions

In [4]:
questions = db.get_questions()
len(questions)

168

Distribution of answer lengths

In [ ]:
df = pd.DataFrame(questions, columns=["question"])

# Topic Modeling

In [3]:
questions = [q.text for q in db.get_questions()]

In [4]:
questions[:5]

['Checking the financial stability of our suppliers is part of our due diligence process. To support us in doing this, please tell us when your company financial year runs to and from.',
 'How many candidates do you have in your database?',
 'How do you select which candidates are placed where?  For instance, if two buyers are looking for the same candidate, how do you ultimately determine which company to place that employee with?',
 'How often do you review and refresh your database of candidates?',
 'What type of background and drug testing do you conduct?  How extensive are they? Who is your service provider for background checking?']

In [5]:
# Fine-tune topic representations with GPT
openai.api_key = os.environ["OPENAI_API_KEY"]
representation_model = OpenAI(model="gpt-3.5-turbo", chat=True)

# representation_model = KeyBERTInspired()
topic_model = BERTopic(representation_model=representation_model)
topics, probs = topic_model.fit_transform(questions)

In [6]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,82,-1_Document management and security processes,[Document management and security processes],[is there formal user registration process inc...
1,0,30,0_Security and Data Privacy Policies and Proce...,[Security and Data Privacy Policies and Proces...,[Is there a formal and well defined security a...
2,1,24,"1_On-site coordinators, turnover rate, fill ra...","[On-site coordinators, turnover rate, fill rat...",[If you were a provider of ours last season wh...
3,2,18,2_Data Security and Access Controls in Segra O...,[Data Security and Access Controls in Segra Or...,[Is there remote access to the network and sys...
4,3,14,3_Access Control and User Authorization in Sec...,[Access Control and User Authorization in Secu...,[Are critical system files and folders protect...


In [7]:
for _, row in topic_model.get_topic_info().iterrows():
    print(row["Name"], row["Count"])
    for doc in row["Representative_Docs"]:
        print(f"\t-{doc.strip()}")
    print()

-1_Document management and security processes 82
	-is there formal user registration process include using unique user IDÕs so that users can be linked to and made responsible for their actions and the use of group IDÕs should only be permitted where they are suitable for the work carried out. Please explain
	-Is there a process in place to ensure all employees and external users return the organization's assets on termination of their employment, contract or agreement?
	-Is there procedures in place for handling secure disposal and or return of the customer data upon request or in the case of termination of services? Please explain

0_Security and Data Privacy Policies and Processes 30
	-Is there a formal and well defined security and privacy related reporting process to Management and oversight groups? If yes, please explain the process?
	-Is  a regulatory data privacy and data security compliance assessment against the applicable global and domestic data privacy and data security re